In [1]:
#all necessary imports
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
df = pd.read_csv("NFL_Team_Data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  672 non-null    int64  
 1   team                  672 non-null    object 
 2   wins                  672 non-null    int64  
 3   losses                672 non-null    int64  
 4   win_loss_perc         672 non-null    float64
 5   points                672 non-null    int64  
 6   points_opp            672 non-null    int64  
 7   points_diff           672 non-null    int64  
 8   mov                   352 non-null    float64
 9   g                     672 non-null    int64  
 10  total_yards           672 non-null    int64  
 11  plays_offense         672 non-null    int64  
 12  yds_per_play_offense  672 non-null    float64
 13  turnovers             672 non-null    int64  
 14  fumbles_lost          672 non-null    int64  
 15  first_down            6

In [2]:
# additional data
df2 = pd.read_csv("Additional Data.csv")
df2

,year,SB,winner,Winner Pts,Loser,Loser Pts,MVP,Stadium,City,State
0,2024,LVIII (58),Kansas City Chiefs,25,San Francisco 49ers,22,Patrick Mahomes,Alleigent Stadium,Las Vegas,Nevada
1,2023,LVII (57),Kansas City Chiefs,38,Philadelphia Eagles,35,Patrick Mahomes,State Farm Stadium,Glendale,Arizona
2,2021,LVI (56),Los Angeles Rams,23,Cincinnati Bengals,20,Cooper Kupp,SoFi Stadium,Inglewood,California
3,2021,LV (55),Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Tom Brady,Raymond James Stadium,Tampa,Florida
4,2020,LIV (54),Kansas City Chiefs,31,San Francisco 49ers,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida
5,2019,LIII (53),New England Patriots,13,Los Angeles Rams,3,Julian Edelman,Mercedes-Benz Stadium,Atlanta,Georgia
6,2018,LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
7,2017,LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
8,2016,50,Denver Broncos,24,Carolina Panthers,10,Von Miller,Levi's Stadium,Santa Clara,California
9,2015,XLIX (49),New England Patriots,28,Seattle Seahawks,24,Tom Brady,University of Phoenix Stadium,Glendale,Arizona


In [3]:
#renaming teams that have changed
df.team.replace({"St. Louis Rams": "Los Angeles Rams",
          "Washington Redskins": "Washington Commanders",
          "Oakland Raiders": "Las Vegas Raiders"},
          inplace = True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_19256\2517370638.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.team.replace({"St. Louis Rams": "Los Angeles Rams",


In [4]:
# joining datasets together
final_data = pd.merge(df,df2,how = "outer", on=["year"])
final_data= final_data.drop(columns=[
    'mov','g', 'MVP','Stadium','City','State','first_down', 'turnovers','pass_net_yds_per_att', 'SB',
    'fumbles_lost','exp_pts_tot', 'points_diff', 'plays_offense','pass_int','pass_att', 'win_loss_perc', 'penalties', 'penalties_yds',
    'pass_cmp','yds_per_play_offense','pass_fd', 'rush_att','rush_yds_per_att','rush_fd','score_pct','turnover_pct','pen_fd',
    'ties'])
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         705 non-null    int64  
 1   team         704 non-null    object 
 2   wins         704 non-null    float64
 3   losses       704 non-null    float64
 4   points       704 non-null    float64
 5   points_opp   704 non-null    float64
 6   total_yards  704 non-null    float64
 7   pass_yds     704 non-null    float64
 8   pass_td      704 non-null    float64
 9   rush_yds     704 non-null    float64
 10  rush_td      704 non-null    float64
 11  winner       673 non-null    object 
 12  Winner Pts   673 non-null    float64
 13  Loser        673 non-null    object 
 14  Loser Pts    673 non-null    float64
dtypes: float64(11), int64(1), object(3)
memory usage: 82.7+ KB


In [5]:

X, y = make_classification(
    n_features = 14,
    n_classes=14,
    n_redundant= 2,
    n_samples= 705,
    n_informative=12,
    n_clusters_per_class=1,
    random_state=1,
    class_sep = 3.5
)

X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.2, random_state=306, shuffle=True)

In [6]:

predictor = GaussianNB()

predictor.fit(X_train, y_train)

prediction = predictor.predict(X_test)

results_dict = {'Accuracy':0,'Precision':0,'Recall':0}

accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction, average='macro')
recall = recall_score(y_test, prediction, average = 'macro')

results_dict.update({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall})
result = pd.DataFrame([results_dict])
result

,Accuracy,Precision,Recall
0,0.992908,0.994898,0.994505
